In [17]:
import random
import numpy as np
import sys
import os
import copy
from collections import Counter
from deap import base, creator, tools
import torch
import torch.nn.functional as F

# --- Device & Grid setup ---
DEVICE = 'cuda'
IND_ROWS, IND_COLS = 8, 14
IND_SIZE = IND_ROWS * IND_COLS
TARGET_POP = 300
FILENAME = 'best_output_double.txt'
MAX_N = 50000

KERNEL = torch.ones((1, 1, 3, 3), device=DEVICE)
KERNEL[0, 0, 1, 1] = 0

creator.create("FitnessMax", base.Fitness, weights=(1.0, 1.0))
creator.create("Individual", list, fitness=creator.FitnessMax)
toolbox = base.Toolbox()

# --- [불러오기 로직] 기존 파일에서 우수한 개체 로드 ---
def load_previous_best():
    loaded = []
    if os.path.exists(FILENAME):
        with open(FILENAME, 'r') as f:
            # 14자리 숫자이고 길이 8줄인 블록 찾기
            valid_lines = [line.strip() for line in f if len(line.strip()) == 14 and line.strip().isdigit()]
        
        for block_start in range(0, len(valid_lines), 8):
            if block_start + 7 >= len(valid_lines): break
            block = valid_lines[block_start:block_start + 8]
            ind_list = []
            for row in block:
                ind_list.extend(int(d) for d in row)
            
            if len(ind_list) == IND_SIZE:
                ind = creator.Individual(ind_list)
                loaded.append(ind)
    
    # 중복 제거 및 최신 순 정렬된 상태라고 가정 (필요시 selBest 등으로 필터링 가능)
    return loaded

# --- GPU 가속 연산부 (생략 없음) ---
def check_paths_multi_batch(grids, num_strs):
    B = grids.shape[0]
    num_count = len(num_strs)
    results = torch.zeros((B, num_count), dtype=torch.bool, device=DEVICE)
    for i, n_str in enumerate(num_strs):
        digits = [int(d) for d in n_str]
        mask = (grids == digits[0]).float()
        for d in digits[1:]:
            if not mask.any():
                mask = None
                break
            spread = F.conv2d(mask, KERNEL, padding=1) > 0
            mask = (spread & (grids == d)).float()
        if mask is not None:
            results[:, i] = mask.view(B, -1).sum(dim=1) > 0
    return results

def evaluate_batch_gpu(ind_lists):
    if not ind_lists: return [], []
    grids = torch.tensor(ind_lists, dtype=torch.int8, device=DEVICE).view(-1, 1, IND_ROWS, IND_COLS)
    B = grids.shape[0]
    current_scores = torch.zeros(B, device=DEVICE)
    still_alive = torch.ones(B, dtype=torch.bool, device=DEVICE)
    formable_counts = torch.zeros(B, device=DEVICE)

    for n in range(1, MAX_N + 1):
        if not still_alive.any(): break
        exists = check_paths_multi_batch(grids, [str(n)])[:, 0]
        current_scores = torch.where(exists & still_alive, current_scores + 1, current_scores)
        still_alive &= exists

    all_nums = [str(n) for n in range(1000, 10000)]
    batch_size = 100 
    for i in range(0, len(all_nums), batch_size):
        batch_strs = all_nums[i:i+batch_size]
        fwd_results = check_paths_multi_batch(grids, batch_strs)
        rev_results = torch.zeros_like(fwd_results)
        rev_strs = [s[::-1] for s in batch_strs]
        valid_rev_indices = [idx for idx, s in enumerate(rev_strs) if not s.startswith('0') and s != batch_strs[idx]]
        if valid_rev_indices:
            rev_batch_res = check_paths_multi_batch(grids, [rev_strs[idx] for idx in valid_rev_indices])
            for idx, res_idx in enumerate(valid_rev_indices):
                rev_results[:, res_idx] = rev_batch_res[:, idx]
        formable_counts += (fwd_results | rev_results).sum(dim=1).float()
    return current_scores.cpu().numpy(), formable_counts.cpu().numpy()

# --- Crowding Distance & GA 연산 (복구 완료) ---
def _assign_crowding_dist_fallback(front):
    if not front: return
    if len(front) <= 2:
        for ind in front: ind.crowding_dist = float("inf")
        return
    for ind in front: ind.crowding_dist = 0.0
    nobj = len(front[0].fitness.values)
    for m in range(nobj):
        front.sort(key=lambda ind: ind.fitness.values[m])
        front[0].crowding_dist = float("inf")
        front[-1].crowding_dist = float("inf")
        fmin, fmax = front[0].fitness.values[m], front[-1].fitness.values[m]
        denom = fmax - fmin
        if denom == 0: continue
        for i in range(1, len(front) - 1):
            front[i].crowding_dist += (front[i+1].fitness.values[m] - front[i-1].fitness.values[m]) / denom

def update_crowding(population):
    fronts = tools.sortNondominated(population, k=len(population), first_front_only=False)
    assign = getattr(tools.emo, "assignCrowdingDist", _assign_crowding_dist_fallback)
    for front in fronts:
        assign(front)
    return [ind for front in fronts for ind in front]

def custom_mate(ind1, ind2):
    grid1, grid2 = np.array(ind1).reshape(IND_ROWS, IND_COLS), np.array(ind2).reshape(IND_ROWS, IND_COLS)
    sy, sx = random.randint(0, IND_ROWS-2), random.randint(0, IND_COLS-2)
    ey, ex = random.randint(sy+1, IND_ROWS), random.randint(sx+1, IND_COLS)
    grid1[sy:ey, sx:ex], grid2[sy:ey, sx:ex] = grid2[sy:ey, sx:ex].copy(), grid1[sy:ey, sx:ex].copy()
    ind1[:], ind2[:] = grid1.flatten().tolist(), grid2.flatten().tolist()
    return ind1, ind2

def custom_mutate(individual):
    if random.random() < 0.5:
        tools.mutUniformInt(individual, 0, 9, 0.05)
    else:
        a, b = random.sample(range(10), 2)
        for i in range(len(individual)):
            if individual[i] == a: individual[i] = b
            elif individual[i] == b: individual[i] = a
    return individual,

toolbox.register("attr_int", random.randint, 0, 9)
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_int, IND_SIZE)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.register("mate", custom_mate)
toolbox.register("mutate", custom_mutate)
toolbox.register("select", tools.selTournamentDCD)

# --- 메인 실행부 ---
def main():
    # 1. 파일에서 데이터 로드 및 초기 인구 구성
    loaded_inds = load_previous_best()
    pop = []
    if loaded_inds:
        print(f"Loaded {len(loaded_inds)} individuals from {FILENAME}")
        pop.extend(loaded_inds)
    
    # 부족한 인구는 랜덤하게 채움
    if len(pop) < TARGET_POP:
        pop.extend(toolbox.population(n=TARGET_POP - len(pop)))
    pop = pop[:TARGET_POP]

    best_ind = None
    max_score_all_time = -1.0

    # 2. 초기 평가
    scores, forms = evaluate_batch_gpu([list(ind) for ind in pop])
    for ind, s, f in zip(pop, scores, forms):
        ind.fitness.values = (s, f)
    pop = update_crowding(pop)

    print(f"H100 NVL Optimization Active. Target Population: {TARGET_POP}")

    for g in range(1, 101):
        offspring = toolbox.select(pop, len(pop))
        offspring = [copy.deepcopy(ind) for ind in offspring]

        for c1, c2 in zip(offspring[::2], offspring[1::2]):
            if random.random() < 0.5:
                toolbox.mate(c1, c2)
                del c1.fitness.values, c2.fitness.values
        
        for m in offspring:
            if random.random() < 0.2:
                toolbox.mutate(m)
                del m.fitness.values

        invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
        if invalid_ind:
            i_scores, i_forms = evaluate_batch_gpu([list(ind) for ind in invalid_ind])
            for ind, s, f in zip(invalid_ind, i_scores, i_forms):
                ind.fitness.values = (s, f)

        combined = pop + offspring
        pop = update_crowding(combined)[:TARGET_POP]

        fits = np.array([ind.fitness.values for ind in pop])
        idx_max = np.argmax(fits[:, 0])
        if fits[idx_max, 0] > max_score_all_time:
            max_score_all_time = fits[idx_max, 0]
            best_ind = copy.deepcopy(pop[idx_max])

        if g % 10 == 0:
            sys.stdout.write(f"-- Gen {g:3d} -- Max Score: {fits[idx_max, 0]:7.0f} | Max Form: {np.max(fits[:, 1]):7.0f}\n")

    if best_ind:
        grid = np.array(best_ind).reshape(IND_ROWS, IND_COLS)
        with open(FILENAME, 'a') as f:
            for row in grid: f.write(''.join(map(str, row)) + '\n')
            f.write(f"Score: {best_ind.fitness.values[0]}, Form: {best_ind.fitness.values[1]}\n\n")
        print(f"Final Best Score: {max_score_all_time}")

if __name__ == "__main__":
    main()

Loaded 41 individuals from best_output_double.txt
H100 NVL Optimization Active. Target Population: 300
-- Gen  10 -- Max Score:    3654 | Max Form:    8960
-- Gen  20 -- Max Score:    2955 | Max Form:    8960
-- Gen  30 -- Max Score:    2907 | Max Form:    8960


KeyboardInterrupt: 